In [1]:
import livef1
session = livef1.get_session(
   season=2024,
   meeting_identifier="Spa",
   session_identifier="Race"
   )

2025-02-03 16:45:43,797 : livef1 : DEBUG :: Trying to get season 2024.
2025-02-03 16:45:43,800 : livef1 : DEBUG :: Sending GET request to URL: https://livetiming.formula1.com/static/2024/Index.json
2025-02-03 16:45:44,649 : livef1 : DEBUG :: Sending GET request to URL: https://livetiming.formula1.com/static/2024/Index.json
2025-02-03 16:45:45,521 : livef1 : DEBUG :: The season was received successfully.
2025-02-03 16:45:45,532 : livef1 : DEBUG :: Trying to get meeting by meeting identifier.
2025-02-03 16:45:45,535 : livef1 : DEBUG :: Searching of identifier 'Spa' has started.
2025-02-03 16:45:45,543 : livef1 : INFO :: Selected meeting/session is:
	Meeting Offname : FORMULA 1 ROLEX BELGIAN GRAND PRIX 2024
	Meeting Name : Belgian Grand Prix
	Meeting Circuit Shortname : Spa-Francorchamps
2025-02-03 16:45:45,546 : livef1 : INFO :: The meeting was received successfully.
2025-02-03 16:45:45,550 : livef1 : DEBUG :: Trying to get session by identifier.
2025-02-03 16:45:45,550 : livef1 : DEBUG 

In [2]:
from time import time
import pandas as pd
import numpy as np
from datetime import timedelta

from livef1.utils.helper import to_datetime
from livef1.utils.constants import interpolation_map

class elapser:
    def __init__(self):
        self.last_times = {}
        self.records = {}

    def set(self, label):
        if label not in self.records:
            self.records[label] = []
        self.last_times[label] = time()

    def calc(self, label):
        self.records[label].append(time() - self.last_times[label])

def generate_laps_table(bronze_lake):
    timer = elapser()

    df_exp = bronze_lake.get("TimingData")

    sector_cols = {
        "Sectors_0_Value": "sector1_time",
        "Sectors_1_Value": "sector2_time",
        "Sectors_2_Value": "sector3_time",
        "Sectors_0_PreviousValue": None,
        "Sectors_1_PreviousValue": None,
        "Sectors_2_PreviousValue": None
    }

    speedTrap_cols = {
        "Speeds_I1_Value": "speed_I1",
        "Speeds_I2_Value": "speed_I2",
        "Speeds_FL_Value": "speed_FL",
        "Speeds_ST_Value": "speed_ST",
    }
    pit_cols = {
        "InPit": "in_pit",
        "PitOut": "pit_out"
    }

    base_cols = {
        "NumberOfLaps": "lap_number",
        "LastLapTime_Value": "lap_time"
    }

    extra_cols = ["no_pits"]
    extra_raw_cols = ["Stopped"]

    col_map = {**base_cols, **pit_cols, **sector_cols, **speedTrap_cols}
    cols = list(base_cols.values()) + list(pit_cols.values()) + list(sector_cols.values()) + list(speedTrap_cols.values())
    raw_cols = list(base_cols.keys()) + list(pit_cols.keys()) + list(sector_cols.keys()) + list(speedTrap_cols.keys()) + extra_raw_cols

    def str_timedelta(x):
        if isinstance(x, str):
            count_sep = x.count(":")
            if count_sep == 0:
                return "00:00:" + x
            elif count_sep == 1:
                return "00:" + x
            else:
                return x
        else:
            return x

    all_laps = []

    for driver_no in df_exp["DriverNo"].unique():
        timer.set("DriverLoop")

        df_driver = df_exp[df_exp["DriverNo"] == driver_no]
        df_test = df_driver[["timestamp"] + raw_cols].dropna(subset=raw_cols, how="all").replace('', np.nan)

        timer.set("to_timedelta Sectors")
        for col in ["Sectors_0_Value", "Sectors_1_Value", "Sectors_2_Value", "Sectors_0_PreviousValue", "Sectors_1_PreviousValue", "Sectors_2_PreviousValue", "LastLapTime_Value"]:
            df_test[col] = df_test[col]
            df_test[col] = pd.to_timedelta(df_test[col].apply(str_timedelta))
        timer.calc("to_timedelta Sectors")


        def enter_new_lap(laps, record):
            if laps is None and record is None:
                no_pits = 0
                laps = []
                record = {key: None if key != "lap_number" else 1 for key in cols}
                record["no_pits"] = no_pits
                return [], record, timedelta(seconds=0)

            if (record["lap_time"] is None) & ((record["sector1_time"] != None) and (record["sector2_time"] != None) and (record["sector3_time"] != None)):
                record["lap_time"] = record["sector1_time"] + record["sector2_time"] + record["sector3_time"]

            laps.append(record)

            no_pits = record["no_pits"]
            record = {key: None if key != "lap_number" else val + 1 for key, val in record.items()}
            record["no_pits"] = no_pits

            return laps, record

        new_lap_allowed = True
        laps, record, last_record_ts = enter_new_lap(None, None)

        timer.set("row iters all")
        for idx, row in df_test.iterrows():
            timer.set("row iter")
            ts = pd.to_timedelta(row.timestamp)

            if row.Stopped == True:
                laps, record = enter_new_lap(laps, record)
                continue

            if not pd.isnull(row.LastLapTime_Value):
                if not pd.isnull(row.Sectors_2_Value):
                    record[col_map["LastLapTime_Value"]] = row.LastLapTime_Value
                elif not pd.isnull(row.Sectors_2_PreviousValue):
                    laps[-1][col_map["LastLapTime_Value"]] = row.LastLapTime_Value

            timer.set("old")
            # timer.set("speedTrap_cols")
            # for sc_key, sc_value in row[list(speedTrap_cols.keys())].dropna().to_dict().items():
            #     record[col_map[sc_key]] = sc_value
            # timer.calc("speedTrap_cols")


            # timer.set("pit_cols")
            # for sc_key, sc_value in row[list(pit_cols.keys())].dropna().to_dict().items():
            #     if sc_key == "InPit":
            #         if sc_value == 1:
            #             record[col_map[sc_key]] = ts
            #     elif sc_key == "PitOut":
            #         if sc_value == True:
            #             record[col_map[sc_key]] = ts
            #             record["no_pits"] += 1
            # timer.calc("pit_cols")

            # timer.set("sector_cols")
            # for sc_key, sc_value in row[list(sector_cols.keys())].dropna().to_dict().items():
            #     sc_no = int(sc_key.split("_")[1])
            #     key_type = sc_key.split("_")[2]

            #     if key_type == "Value":
            #         if record[f"sector{str(sc_no + 1)}_time"] == None:
            #             record[f"sector{str(sc_no + 1)}_time"] = sc_value
            #             last_record_ts = ts
            #             if sc_no == 2:
            #                 laps, record = enter_new_lap(laps, record)
            #                 record["lap_start_time"] = ts
            #         elif sc_value == record[str(sc_no + 1)]:
            #             pass
            #         elif ts - last_record_ts > timedelta(seconds=10):
            #             laps, record = enter_new_lap(laps, record)
            #             record[f"sector{str(sc_no + 1)}_time"] = sc_value
            #             last_record_ts = ts

            #     elif key_type == "PreviousValue" and ts - last_record_ts > timedelta(seconds=10):
            #         record[f"sector{str(sc_no + 1)}_time"] = sc_value
            #         last_record_ts = ts
            #         if sc_no == 2:
            #             laps, record = enter_new_lap(laps, record)
            # timer.calc("sector_cols")
            timer.calc("old")
            
            timer.set("new")
            for sc_key, sc_value in row.to_dict().items():
                if not pd.isna(sc_value):
                    if sc_key in speedTrap_cols:
                        record[col_map[sc_key]] = sc_value
                    
                    if sc_key in pit_cols:
                        if sc_key == "InPit":
                            if sc_value == 1:
                                record[col_map[sc_key]] = ts
                        elif sc_key == "PitOut":
                            if sc_value == True:
                                record[col_map[sc_key]] = ts
                                record["no_pits"] += 1
                    if sc_key in sector_cols:
                        sc_no = int(sc_key.split("_")[1])
                        key_type = sc_key.split("_")[2]

                        if key_type == "Value":
                            if record[f"sector{str(sc_no + 1)}_time"] == None:
                                record[f"sector{str(sc_no + 1)}_time"] = sc_value
                                last_record_ts = ts
                                if sc_no == 2:
                                    laps, record = enter_new_lap(laps, record)
                                    record["lap_start_time"] = ts
                            elif sc_value == record[str(sc_no + 1)]:
                                pass
                            elif ts - last_record_ts > timedelta(seconds=10):
                                laps, record = enter_new_lap(laps, record)
                                record[f"sector{str(sc_no + 1)}_time"] = sc_value
                                last_record_ts = ts

                        elif key_type == "PreviousValue" and ts - last_record_ts > timedelta(seconds=10):
                            record[f"sector{str(sc_no + 1)}_time"] = sc_value
                            last_record_ts = ts
                            if sc_no == 2:
                                laps, record = enter_new_lap(laps, record)
            timer.calc("new")
            timer.calc("row iter")
        timer.calc("row iters all")
                  

        laps_df = pd.DataFrame(laps)
        laps_df["DriverNo"] = driver_no
        all_laps.append(laps_df)
        timer.calc("DriverLoop")


    all_laps_df = pd.concat(all_laps, ignore_index=True)

    timer.set("fill_missing")
    segments = ["sector1_time", "sector2_time", "sector3_time"]
    for idx in range(len(segments)):
        rest = np.delete(segments, idx)
        all_laps_df[segments[idx]] = (all_laps_df[segments[idx]].fillna(timedelta(minutes=0)) + (all_laps_df[segments[idx]].isnull() & (all_laps_df["lap_number"] > 1)) * (all_laps_df[segments[idx]].isnull() * (all_laps_df["lap_time"].fillna(timedelta(minutes=0)) - all_laps_df[rest].sum(axis=1)))).replace(timedelta(minutes=0), np.timedelta64("NaT"))
    timer.calc("fill_missing")

    new_ts = (all_laps_df["lap_start_time"] + all_laps_df["lap_time"]).shift(1)
    all_laps_df["lap_start_time"] = (new_ts.isnull() * all_laps_df["lap_start_time"]) + new_ts.fillna(timedelta(0))
    all_laps_df["lap_start_date"] = (all_laps_df["lap_start_time"] + bronze_lake.great_lake.session.first_datetime).fillna(bronze_lake.great_lake.session.session_start_datetime)

    all_laps_df[["lap_start_time"] + all_laps_df.columns.tolist()]
    return all_laps_df, timer

In [3]:
from livef1.utils.logger import logger
from livef1.utils.constants import TOPICS_MAP, SILVER_SESSION_TABLES, TABLE_GENERATION_FUNCTIONS
from livef1.utils.constants import TABLE_GENERATION_FUNCTIONS, TABLE_REQUIREMENTS

def generate_table(silver_lake, table_name):
    """
    Generate a table using the corresponding function from silver_functions.py.

    Parameters
    ----------
    table_name : str
        The name of the table to generate.

    Returns
    -------
    DataFrame
        The generated table as a pandas DataFrame.
    """
    self = silver_lake
    if table_name in TABLE_GENERATION_FUNCTIONS:
        required_data = TABLE_REQUIREMENTS.get(table_name, [])
        for data_name in required_data:
            if not self.bronze_lake.has_data(data_name):
                self.great_lake.session.load_data(data_name)
        function_name = TABLE_GENERATION_FUNCTIONS[table_name]
        return globals()[function_name](self.bronze_lake)
    else:
        raise ValueError(f"No generation function found for table: {table_name}")


def generate(sess, silver=True, gold=False):
    timer = elapser()
    self = sess
    timer.set("initials")
    self.first_datetime = self._get_first_datetime()
    self.session_start_time = self._get_session_start_time()
    self.session_start_datetime = self.first_datetime + self.session_start_time
    timer.calc("initials")

    if silver:
        logger.info(f"Silver tables are being generated.")
        for table_name in SILVER_SESSION_TABLES:
            if table_name in TABLE_GENERATION_FUNCTIONS:
                setattr(self, table_name, generate_table(self.data_lake.silver_lake, table_name))
                logger.info(f"'{table_name}' has been generated and saved to the silver lake. You can access it from 'session.{table_name}'.")

    if gold:
        pass

In [4]:
session = livef1.get_session(
   season=2024,
   meeting_identifier="Spa",
   session_identifier="Race"
   )

generate(session)

2025-02-03 16:45:56,499 : livef1 : DEBUG :: Trying to get season 2024.
2025-02-03 16:45:56,502 : livef1 : DEBUG :: Sending GET request to URL: https://livetiming.formula1.com/static/2024/Index.json
2025-02-03 16:45:57,219 : livef1 : DEBUG :: Sending GET request to URL: https://livetiming.formula1.com/static/2024/Index.json
2025-02-03 16:45:57,983 : livef1 : DEBUG :: The season was received successfully.
2025-02-03 16:45:57,988 : livef1 : DEBUG :: Trying to get meeting by meeting identifier.
2025-02-03 16:45:57,989 : livef1 : DEBUG :: Searching of identifier 'Spa' has started.
2025-02-03 16:45:57,990 : livef1 : INFO :: Selected meeting/session is:
	Meeting Offname : FORMULA 1 ROLEX BELGIAN GRAND PRIX 2024
	Meeting Name : Belgian Grand Prix
	Meeting Circuit Shortname : Spa-Francorchamps
2025-02-03 16:45:57,990 : livef1 : INFO :: The meeting was received successfully.
2025-02-03 16:45:57,990 : livef1 : DEBUG :: Trying to get session by identifier.
2025-02-03 16:45:57,999 : livef1 : DEBUG 

<class 'pandas.core.frame.DataFrame'>


2025-02-03 16:46:18,971 : livef1 : INFO :: 'SessionStatus' has not been found in lake, loading it.
2025-02-03 16:46:18,973 : livef1 : INFO :: Getting requested data : 'SessionStatus'.
	Selected session : 2024 Belgian Grand Prix Race
	Topic : SessionStatus
2025-02-03 16:46:18,973 : livef1 : DEBUG :: Sending GET request to URL: https://livetiming.formula1.com/static/2024/2024-07-28_Belgian_Grand_Prix/2024-07-28_Race/SessionStatus.jsonStream
2025-02-03 16:46:19,672 : livef1 : DEBUG :: Successfully parsed streamed data.
2025-02-03 16:46:19,675 : livef1 : DEBUG :: Data has been get in 0.701 seconds
2025-02-03 16:46:19,677 : livef1 : INFO :: Data is successfully received.
2025-02-03 16:46:19,679 : livef1 : DEBUG :: Using parser function for title: SessionStatus
2025-02-03 16:46:19,682 : livef1 : DEBUG :: Parsing successful for title: SessionStatus
2025-02-03 16:46:19,686 : livef1 : DEBUG :: Data has been parsed in 0.007 seconds
2025-02-03 16:46:19,689 : livef1 : INFO :: Data is successfully 

KeyError: 'generate_car_telemetry_table'

In [19]:
from livef1.utils.helper import *
from livef1.utils.constants import channel_name_map
from livef1.data_processing.etl import livef1SessionETL

def parse_position_z(data, sessionKey):
    """
    Parses driver position (z-axis) data.

    Parameters
    ----------
        data : :class:`dict`
            The driver position data.
        sessionKey : :class:`int`
            The key of the current session.

    Yields
    ----------
        dict :
            A record containing the session key, timestamp, UTC time, driver number, and z-axis position data.
    """
    for ts, v in data:
    #data.items()::
        parsed_entry = parse(v, zipped=True)
        for position_entry in parsed_entry["Position"]:
            utc = position_entry["Timestamp"]
            for driver_entry in position_entry["Entries"].items():
                record = {
                    "SessionKey": sessionKey,
                    "timestamp": ts,
                    "Utc": utc,
                    "DriverNo": driver_entry[0],
                    **driver_entry[1]
                }
                yield record

def parse(text: str, zipped: bool = False) -> Union[str, dict]:
    """
    Parses a given text input and decompresses it if necessary.

    Parameters
    ----------
    text : str
        The input text to be parsed.
    zipped : bool, optional
        Indicates if the input is a zipped string, by default False.

    Returns
    -------
    Union[str, dict]
        The parsed output as a dictionary if input is JSON, otherwise as a string.
    """
    if text[0] == '{':  # Check if the text is in JSON format.
        return json.loads(text)  # Return parsed JSON as a dictionary.
    if text[0] == '"':  # Check if the text is a quoted string.
        text = text.strip('"')  # Remove surrounding quotes.
    if zipped:
        # Decompress the zipped base64 string and parse it.
        text = zlib.decompress(base64.b64decode(text), -zlib.MAX_WBITS)
        return parse(text.decode('utf-8-sig'))
    return text  # Return the text as is if it's not zipped.

s = time()
data = livef1.adapters.livetimingF1_getdata(
    "https://livetiming.formula1.com/static/2024/2024-07-28_Belgian_Grand_Prix/2024-07-28_Race/Position.z.jsonStream",
    True
)
print(time() - s)

s = time()
data = [i for i in livef1SessionETL(session=session).unified_parse("Position.z", data)]
df = pd.DataFrame(data)
print(time() - s)

2025-02-03 17:06:48,550 : livef1 : DEBUG :: Sending GET request to URL: https://livetiming.formula1.com/static/2024/2024-07-28_Belgian_Grand_Prix/2024-07-28_Race/Position.z.jsonStream
2025-02-03 17:06:54,721 : livef1 : DEBUG :: Successfully parsed streamed data.
2025-02-03 17:06:54,886 : livef1 : DEBUG :: Using parser function for title: Position.z
2025-02-03 17:06:54,889 : livef1 : DEBUG :: Parsing successful for title: Position.z


6.335913896560669
3.441941261291504


In [18]:
session.load_data("Position")

2025-02-03 17:06:34,866 : livef1 : INFO :: Getting requested data : 'Position.z'.
	Selected session : 2024 Belgian Grand Prix Race
	Topic : Position.z
2025-02-03 17:06:34,868 : livef1 : DEBUG :: Sending GET request to URL: https://livetiming.formula1.com/static/2024/2024-07-28_Belgian_Grand_Prix/2024-07-28_Race/Position.z.jsonStream
2025-02-03 17:06:38,735 : livef1 : DEBUG :: Successfully parsed streamed data.
2025-02-03 17:06:38,740 : livef1 : DEBUG :: Data has been get in 3.873 seconds
2025-02-03 17:06:38,740 : livef1 : INFO :: Data is successfully received.
2025-02-03 17:06:38,740 : livef1 : DEBUG :: Using parser function for title: Position.z
2025-02-03 17:06:38,751 : livef1 : DEBUG :: Parsing successful for title: Position.z
2025-02-03 17:06:41,802 : livef1 : DEBUG :: Data has been parsed in 3.062 seconds
2025-02-03 17:06:41,802 : livef1 : INFO :: Data is successfully parsed.


,SessionKey,timestamp,Utc,DriverNo,Status,X,Y,Z
0,9574,00:01:45.570,2024-07-28T12:10:22.7877313Z,1,OnTrack,0,0,0
1,9574,00:01:45.570,2024-07-28T12:10:22.7877313Z,2,OnTrack,0,0,0
2,9574,00:01:45.570,2024-07-28T12:10:22.7877313Z,3,OnTrack,0,0,0
3,9574,00:01:45.570,2024-07-28T12:10:22.7877313Z,4,OnTrack,0,0,0
4,9574,00:01:45.570,2024-07-28T12:10:22.7877313Z,10,OnTrack,0,0,0
...,...,...,...,...,...,...,...,...
625015,9574,02:17:07.353,2024-07-28T14:25:45.3300945Z,44,OnTrack,1050,2542,4143
625016,9574,02:17:07.353,2024-07-28T14:25:45.3300945Z,55,OnTrack,1689,1810,4126
625017,9574,02:17:07.353,2024-07-28T14:25:45.3300945Z,63,OnTrack,1292,2288,4137
625018,9574,02:17:07.353,2024-07-28T14:25:45.3300945Z,77,OnTrack,-818,3970,4141


In [122]:
df, timer = generate_laps_table(session.data_lake.bronze_lake)

C:\Users\EU10698\AppData\Local\Temp\ipykernel_11488\2724216116.py:224: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_laps_df = pd.concat(all_laps, ignore_index=True)


In [105]:
{label: np.mean(vals) for label, vals in timer.records.items()}

{'DriverLoop': 0.2488957166671753,
 'to_timedelta Sectors': 0.01305161714553833,
 'row iters all': 0.19765708446502686,
 'row iter': 0.0005495899584785236,
 'old': 9.760209515619311e-07,
 'new': 0.0004485795316506609,
 'fill_missing': 0.014463663101196289}

In [125]:
{label: np.sum(vals) for label, vals in timer.records.items()}

{'DriverLoop': 2.77744197845459,
 'to_timedelta Sectors': 0.2790687084197998,
 'row iters all': 1.7131402492523193,
 'row iter': 1.035132884979248,
 'old': 0.003507852554321289,
 'new': 0.41164565086364746,
 'fill_missing': 0.018542051315307617}

In [123]:
df

,lap_number,lap_time,in_pit,pit_out,sector1_time,sector2_time,sector3_time,None,speed_I1,speed_I2,speed_FL,speed_ST,no_pits,lap_start_time,DriverNo,lap_start_date
0,1,NaT,0 days 00:17:07.661000,NaT,NaT,0 days 00:00:48.663000,0 days 00:00:29.571000,None,314,204,None,303,0,NaT,16,2024-07-28 13:03:52.742
1,2,0 days 00:01:50.240000,NaT,NaT,0 days 00:00:31.831000,0 days 00:00:48.675000,0 days 00:00:29.734000,None,303,203,219,None,0,0 days 00:57:07.067000,16,2024-07-28 13:05:45.045
2,3,0 days 00:01:50.519000,NaT,NaT,0 days 00:00:31.833000,0 days 00:00:49.132000,0 days 00:00:29.554000,None,311,202,215,304,0,0 days 00:58:57.307000,16,2024-07-28 13:07:35.285
3,4,0 days 00:01:49.796000,NaT,NaT,0 days 00:00:31.592000,0 days 00:00:48.778000,0 days 00:00:29.426000,None,312,201,217,309,0,0 days 01:00:47.870000,16,2024-07-28 13:09:25.848
4,5,0 days 00:01:49.494000,NaT,NaT,0 days 00:00:31.394000,0 days 00:00:48.729000,0 days 00:00:29.371000,None,313,197,217,311,0,0 days 01:02:37.721000,16,2024-07-28 13:11:15.699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,40,0 days 00:01:48.760000,NaT,NaT,0 days 00:00:31.050000,0 days 00:00:48.625000,0 days 00:00:29.085000,None,320,206,221,None,1,0 days 02:07:26.976000,22,2024-07-28 14:16:04.954
838,41,0 days 00:01:48.567000,NaT,NaT,0 days 00:00:31.018000,0 days 00:00:48.511000,0 days 00:00:29.038000,None,316,205,220,None,1,0 days 02:09:15.720000,22,2024-07-28 14:17:53.698
839,42,0 days 00:01:48.574000,NaT,NaT,0 days 00:00:31.073000,0 days 00:00:48.412000,0 days 00:00:29.089000,None,318,209,220,312,1,0 days 02:11:04.237000,22,2024-07-28 14:19:42.215
840,43,0 days 00:01:48.212000,NaT,NaT,0 days 00:00:31.019000,0 days 00:00:48.263000,0 days 00:00:28.930000,None,317,207,220,None,1,0 days 02:12:53.044000,22,2024-07-28 14:21:31.022


In [124]:
session.laps

,lap_number,lap_time,in_pit,pit_out,sector1_time,sector2_time,sector3_time,None,speed_I1,speed_I2,speed_FL,speed_ST,no_pits,lap_start_time,DriverNo,lap_start_date
0,1,NaT,0 days 00:17:07.661000,NaT,NaT,0 days 00:00:48.663000,0 days 00:00:29.571000,None,314,204,219,303,0,NaT,16,2024-07-28 13:03:52.742
1,2,0 days 00:01:50.240000,NaT,NaT,0 days 00:00:31.831000,0 days 00:00:48.675000,0 days 00:00:29.734000,None,303,203,215,None,0,0 days 00:57:07.067000,16,2024-07-28 13:05:45.045
2,3,0 days 00:01:50.519000,NaT,NaT,0 days 00:00:31.833000,0 days 00:00:49.132000,0 days 00:00:29.554000,None,311,202,217,304,0,0 days 00:58:57.307000,16,2024-07-28 13:07:35.285
3,4,0 days 00:01:49.796000,NaT,NaT,0 days 00:00:31.592000,0 days 00:00:48.778000,0 days 00:00:29.426000,None,312,201,217,309,0,0 days 01:00:47.870000,16,2024-07-28 13:09:25.848
4,5,0 days 00:01:49.494000,NaT,NaT,0 days 00:00:31.394000,0 days 00:00:48.729000,0 days 00:00:29.371000,None,313,197,216,311,0,0 days 01:02:37.721000,16,2024-07-28 13:11:15.699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,40,0 days 00:01:48.760000,NaT,NaT,0 days 00:00:31.050000,0 days 00:00:48.625000,0 days 00:00:29.085000,None,320,206,220,None,1,0 days 02:07:26.976000,22,2024-07-28 14:16:04.954
838,41,0 days 00:01:48.567000,NaT,NaT,0 days 00:00:31.018000,0 days 00:00:48.511000,0 days 00:00:29.038000,None,316,205,220,None,1,0 days 02:09:15.720000,22,2024-07-28 14:17:53.698
839,42,0 days 00:01:48.574000,NaT,NaT,0 days 00:00:31.073000,0 days 00:00:48.412000,0 days 00:00:29.089000,None,318,209,220,312,1,0 days 02:11:04.237000,22,2024-07-28 14:19:42.215
840,43,0 days 00:01:48.212000,NaT,NaT,0 days 00:00:31.019000,0 days 00:00:48.263000,0 days 00:00:28.930000,None,317,207,222,None,1,0 days 02:12:53.044000,22,2024-07-28 14:21:31.022
